In [ ]:
%matplotlib inline

from bokeh.plotting import output_notebook, figure, show
from bokeh.io import push_notebook, export_svgs
from bokeh.layouts import column
from ipywidgets import interact
import numpy
import selenium

## Functions for computation f(x,y) and y(x) for Euler method

In [ ]:
def compute_f_euler(x, y):  # f(xi, yi) = sin^2(xi) + yi*ctg(xi)
    return numpy.sin(x)**2 + y * (numpy.cos(x) / numpy.sin(x))

def compute_y_euler(last_y, last_f, h):  # yi+1 = yi + h*f(xi, yi)
    return last_f * h + last_y

## Functions for computation f(x,y) and y(x) for Improved Euler method

In [ ]:
def compute_f_euler_imp(x, y):  # f(xi, yi) = sin^2(xi) + yi*ctg(xi)
    return numpy.sin(x) ** 2 + y * (numpy.cos(x) / numpy.sin(x))

def compute_y_euler_imp(last_y, last_f, h):  # yi+1 = yi + h*f(xi + h/2, yi + h/2 * f(xi, yi))
    return last_f * h + last_y


## Functions for computation f(x,y) and y(x) for Runge-Kutta method

In [ ]:
def compute_f_run(x, y):  # f(xi, yi) = sin^2(xi) + yi*ctg(xi)
    return numpy.sin(x) ** 2 + y * (numpy.cos(x) / numpy.sin(x))

def compute_y_run(last_y, last_f, h):  # yi+1 = yi + h/6 * (k1 + 2*k2 + 2*k3 + k4)
    return last_f * (h / 6) + last_y


## Compute x

In [ ]:
def find_x(x0, X, h):
    return numpy.linspace(x0, X, int((X - x0) / h) + 1)  # Create array of x values with step h


## Exact solution of the given function

In [ ]:
def exact(x0, y0, X, h):
    x = find_x(x0, X, h)
    c = (y0 + numpy.sin(x0) * numpy.cos(x0)) / numpy.sin(x0)  # initial value problem
    y_exact = -numpy.sin(x) * numpy.cos(x) + c * numpy.sin(x)  # Compute y values
    return y_exact


## Computations for Euler method

In [ ]:
def euler(x0, y0, X, h):
    x = find_x(x0, X, h)
    # Step 0
    # Remember last computed value of yi
    last_y = y0

    # Remember last computed value of f
    last_f = compute_f_euler(x0, y0)

    # Create array of y(x) values
    y_euler = [y0]

    # Step 1..n
    for xi in x[1:]:
        # Compute y, remember it for the next step
        last_y = compute_y_euler(last_y, last_f, h)
        y_euler.append(last_y)

        # Compute f, remember it for the next step
        last_f = compute_f_euler(xi, last_y)
    return y_euler


## Computations for Improved Euler method

In [ ]:
def euler_imp(x0, y0, X, h):
    x = find_x(x0, X, h)
    # Step 0
    # Remember last computed value of yi
    last_y_imp = y0

    # Remember last computed value of f
    last_f_imp = compute_f_euler_imp(x0 + (h / 2),
                                     y0 + (h / 2) * compute_f_euler_imp(x0, y0))

    # Create array of y(x) values
    y_euler_imp = [y0]

    # Step 1..n
    for xi in x[1:]:
        # Compute y, remember it for the next step
        last_y_imp = compute_y_euler_imp(last_y_imp, last_f_imp, h)
        y_euler_imp.append(last_y_imp)

        # Compute f, remember it for the next step
        last_f_imp = compute_f_euler_imp(xi + (h / 2),
                                         last_y_imp + (h / 2) * compute_f_euler_imp(xi, last_y_imp))
    return y_euler_imp


## Computations for Runge-Kutta method

In [ ]:
def runge_kutta(x0, y0, X, h):
    x = find_x(x0, X, h)
    # Step 0
    # Remember last computed value of yi
    last_y_run = y0

    # Compute f = k1 + 2*k2 + 2*k3 + k4
    k1 = compute_f_run(x0, y0)
    k2 = compute_f_run(x0 + (h / 2), y0 + ((h * k1) / 2))
    k3 = compute_f_run(x0 + (h / 2), y0 + ((h * k2) / 2))
    k4 = compute_f_run(x0 + h, y0 + h * k3)

    # Remember last computed value of f
    last_f_run = k1 + (2 * k2) + (2 * k3) + k4

    # Create array of y(x) values
    y_run = [y0]

    # Step 1..n
    for xi in x[1:]:
        # Compute y, remember it for the next step
        last_y_run = compute_y_run(last_y_run, last_f_run, h)
        y_run.append(last_y_run)

        # Compute f = k1 + 2*k2 + 2*k3 + k4
        k1 = compute_f_run(xi, last_y_run)
        k2 = compute_f_run(xi + (h / 2), last_y_run + ((h * k1) / 2))
        k3 = compute_f_run(xi + (h / 2), last_y_run + ((h * k2) / 2))
        k4 = compute_f_run(xi + h, last_y_run + h * k3)

        # Compute f, remember it for the next step
        last_f_run = k1 + (2 * k2) + (2 * k3) + k4
    return y_run


## Computation of global errors

In [ ]:
def global_error(x0, y0, X, step0, stepMax, step_of_steps, x_func, y_exact_func, y_euler_func, y_imp_euler_func, y_runge_kutta_func):
    step = numpy.linspace(step0, stepMax, int((stepMax - step0) / step_of_steps) + 1)
    euler_error = []
    imp_euler_error = []
    runge_kutta_error = []
    for i in step:
        y_exact = y_exact_func(x0, y0, X, i)
        y_euler = y_euler_func(x0, y0, X, i)
        y_euler_imp = y_imp_euler_func(x0, y0, X, i)
        y_run = y_runge_kutta_func(x0, y0, X, i)
        euler_error.append(max(abs(y_exact - y_euler)))
        imp_euler_error.append(max(abs(y_exact - y_euler_imp)))
        runge_kutta_error.append(max(abs(y_exact - y_run)))
    return euler_error, imp_euler_error, runge_kutta_error

## Compute results

In [ ]:
x0 = 1
y0 = 1
X = 3
h = 0.1
step0 = 0.0001
stepMax = 1
step_of_steps = 0.0001

x = find_x(x0, X, h)
y_exact = exact(x0, y0, X, h)
y_euler = euler(x0, y0, X, h)
y_euler_imp = euler_imp(x0, y0, X, h)
y_run = runge_kutta(x0, y0, X, h)

steps = numpy.linspace(step0, stepMax, int((stepMax - step0) / step_of_steps) + 1)
euler_error, imp_euler_error, runge_kutta_error = global_error(x0, y0, X, step0, stepMax, step_of_steps, find_x, exact, euler, euler_imp, runge_kutta)


## Compose the plot and errors

In [ ]:
output_notebook()

In [ ]:
p1 = figure(title="Numerical methods", x_axis_label='x', y_axis_label='y')

a = p1.line(x, y_exact, legend="Exact solution", line_width=2, line_color="#FF0000")
b = p1.line(x, y_euler, legend="Euler method", line_width=2, line_color="#00FF00")
c = p1.line(x, y_euler_imp, legend="Improved Euler method", line_width=2, line_color="#0000FF")
d = p1.line(x, y_run, legend="Runge-Kutta method", line_width=2, line_color="#FF00FF")

p2 = figure(title="Errors of methods", x_axis_label='step', y_axis_label='error')

e = p2.line(steps, euler_error, legend="Error of Euler", line_width=2, line_color="#FF0000")
f = p2.line(steps, imp_euler_error, legend="Error of improved Euler", line_width=2, line_color="#00FF00")
g = p2.line(steps, runge_kutta_error, legend="Error of Runge-Kutta", line_width=2, line_color="#0000FF")

p3 = figure(title="Numerical methods", x_axis_label='x', y_axis_label='y')

p3.line(x, y_exact, legend="Exact solution", line_width=2, line_color="#FF0000")
p3.line(x, y_euler, legend="Euler method", line_width=2, line_color="#00FF00")
p3.line(x, y_euler_imp, legend="Improved Euler method", line_width=2, line_color="#0000FF")
p3.line(x, y_run, legend="Runge-Kutta method", line_width=2, line_color="#FF00FF")


p1.legend.location = "bottom_left"
p2.legend.location = "top_left"
p3.legend.location = "top_left"
p1.legend.click_policy = "hide"
p2.output_backend = "svg"
p3.output_backend = "svg"

export_svgs(p3, "Numerical methods.svg", height=2500, width=2500)
export_svgs(p2, "Errors for methods.svg", height=2500, width=2500)

## Update the plot

In [ ]:
def update(InitialX='1', InitialY='1', MaxX='3', Step='0.1'):
    if InitialX == '' or InitialY == '' or MaxX == '' or Step == '':
        return
    x0 = int(InitialX)
    y0 = int(InitialY)
    X = int(MaxX)
    h = float(Step)
    if X <= x0:
        return
    if h <= 0:
        return
    x = find_x(x0, X, h)
    y_exact = exact(x0, y0, X, h)
    y_euler = euler(x0, y0, X, h)
    y_euler_imp = euler_imp(x0, y0, X, h)
    y_run = runge_kutta(x0, y0, X, h)

    a_dict = {'x': x, 'y': y_exact}
    b_dict = {'x': x, 'y': y_euler}
    c_dict = {'x': x, 'y': y_euler_imp}
    d_dict = {'x': x, 'y': y_run}

    a.data_source.data = a_dict
    b.data_source.data = b_dict
    c.data_source.data = c_dict
    d.data_source.data = d_dict

    push_notebook()

In [ ]:
show(p1, notebook_handle=True)

In [ ]:
interact(update, x0='1', y0='1', X='3', h='0.1')